In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from datetime import datetime

import threading
import time
import pandas
from pandas import DataFrame

In [2]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []  # Initialize variable to store candle

    def historicalData(self, reqId, bar):
        print(f'reqId: {reqId} Time: {bar.date} Close: {bar.close}')
        self.data.append([reqId, bar.date, bar.close])

In [3]:
app = IBapi()
app.connect('127.0.0.1', 7497, 10645)


def run_loop():
    app.run()

In [4]:
reqId_serial = 1

In [5]:
# Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
time.sleep(1)  # Sleep interval to allow time for connection to server

ERROR -1 2158 Sec-def data farm connection is OK:secdefil


In [6]:
def create_contract(curr1, curr2):
    # Create contract object
    contract = Contract()
    contract.symbol = curr1
    contract.secType = 'CASH'
    contract.exchange = 'IDEALPRO'
    contract.currency = curr2
    return contract

In [7]:
EUR_USD_contract = create_contract('EUR', 'USD')

In [8]:
# Request historical candles
app.reqHistoricalData(reqId=1, contract=EUR_USD_contract, endDateTime='', durationStr='1 D', barSizeSetting='1 day',
                      whatToShow='MIDPOINT', useRTH=0, formatDate=1, keepUpToDate=False, chartOptions=[])
time.sleep(2)  # sleep to allow enough time for data to be returned

ERROR -1 2106 HMDS data farm connection is OK:cashhmds


reqId: 1 Time: 20220421 Close: 1.08308


In [9]:
df = pandas.DataFrame(app.data, columns=['reqId', 'DateTime', 'Close'])

In [10]:
df['DateTime'] = [datetime.strptime(dt, '%Y%m%d') for dt in df['DateTime']]

In [11]:
def request_data(contract):
    global reqId_serial
    reqId_serial += 1
    app.reqHistoricalData(reqId=reqId_serial, contract=contract, endDateTime='', durationStr='1 D',
                          barSizeSetting='1 day',
                          whatToShow='MIDPOINT', useRTH=0, formatDate=1, keepUpToDate=False, chartOptions=[])
    time.sleep(0.5)
    if app.data[len(app.data) - 1][0] != reqId_serial:
        return float('NaN')
    else:
        return app.data[len(app.data) - 1][2]

In [12]:
print(request_data(EUR_USD_contract))

reqId: 2 Time: 20220421 Close: 1.08308
1.08308


In [13]:
def fetch_exc_rate(base, quote):
    contract = create_contract(base, quote)
    return request_data(contract)

In [14]:
currencies = ['USD', 'EUR', 'GBP', 'JPY', 'AUD']

In [15]:
def fetch_all(curr_list):
    matrix = DataFrame(columns=curr_list, index=curr_list)
    for i in range(0, len(curr_list)):
        for j in range(0, len(curr_list)):
            if i == j:
                matrix[curr_list[i]][curr_list[j]] = 1
            else:
                matrix[curr_list[i]][curr_list[j]] = fetch_exc_rate(curr_list[i], curr_list[j])
    return matrix

In [16]:
exchange_rates = fetch_all(currencies)

ERROR 3 200 No security definition has been found for the request
ERROR 4 200 No security definition has been found for the request


reqId: 5 Time: 20220421 Close: 128.326


ERROR 6 200 No security definition has been found for the request


reqId: 7 Time: 20220421 Close: 1.08308
reqId: 8 Time: 20220421 Close: 0.829975
reqId: 9 Time: 20220421 Close: 138.987
reqId: 10 Time: 20220421 Close: 1.45837
reqId: 11 Time: 20220421 Close: 1.304935


ERROR 12 200 No security definition has been found for the request


reqId: 13 Time: 20220421 Close: 167.4595
reqId: 14 Time: 20220421 Close: 1.75704


ERROR 15 200 No security definition has been found for the request
ERROR 16 200 No security definition has been found for the request
ERROR 17 200 No security definition has been found for the request
ERROR 18 200 No security definition has been found for the request


reqId: 19 Time: 20220421 Close: 0.74269


ERROR 20 200 No security definition has been found for the request
ERROR 21 200 No security definition has been found for the request


reqId: 22 Time: 20220421 Close: 95.3055


In [17]:
exchange_rates

,USD,EUR,GBP,JPY,AUD
USD,1,1.08308,1.304935,NaN,0.74269
EUR,NaN,1,NaN,NaN,NaN
GBP,NaN,0.829975,1,NaN,NaN
JPY,128.326,138.987,167.4595,1,95.3055
AUD,NaN,1.45837,1.75704,NaN,1
